# 機械学習入門〜手書き数字認識をいろんなモデルで試してみる〜

## やりたいこと
数多ある機械学習手法のなかでスタンダードないくつかのモデルを用いて手書き数字データセット(MNIST)を学習し、評価してみる
### まずはライブラリのインポートから

In [ ]:
#coding: utf-8
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score
from sklearn.utils import testing
from keras.datasets import mnist

In [ ]:
#手書き文字のダウンロード
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### ダウンロードしたデータ内訳
x_train: モデルに学習させるデータセット(28*28のグレースケール画像データ)  
y_train: 学習させるデータセットの答え(0~9の整数)  
x_test: 学習を評価するためのデータセット  
y_test: x_testの答え  
### データの形状を確認する

In [ ]:
print("x_train:", x_train.shape)
print("y_train:", y_train.shape)
print("x_test:", x_test.shape)
print("y_test:", y_test.shape)

データ形状からわかるようにMNISTでは学習モデルに与えるデータが60000、学習した結果を評価するために必要なデータが10000ある。  
### なぜ評価用のデータが必要なの？
学習したデータだけでそのモデルを評価してしまうと、与えられたデータだけに特化しているのか、他のデータでも同程度の精度で推論できているかがわからないため  
このように、学習データのみにモデルの推論が特化してしまうことを<b>過学習</b>という。

<img src="MNIST_LT_1.png">
### MNISTの画像を見てみよう
最初に`import matplotlib.pyplot as plt`としたのはグラフ描画ライブラリである`matplotlib`をインポートするため。  
これにより簡単にグラフや画像を表示することができる。  
試しに訓練データの１番目の画像データを表示してみる

In [ ]:
fig = plt.figure(figsize=((6,6)))
for i in range(1, 26):
    ax = fig.add_subplot(5, 5, i)   #5*5に画像を配列したi番目の位置にプロットする
    ax.axis("off")    #目盛りをなくす
    ax.imshow(x_train[i-1], cmap="gray")    #画像を表示

### 学習の前に…データを整える
画像データは0~255の整数値をとる28*28の配列が並んでいる  
これでは範囲が大きすぎてモデルにとって都合が悪い場合があるので、0~1の範囲にすべて収める

In [ ]:
x_train = x_train / 255
x_test = x_test / 255
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
print(x_train.shape)

## 今回試したい学習モデル
- Dicision Tree(決定木)
- Random Forest
- Neural Network

## Dicision Tree(決定木)
ある条件を用いて分岐していき、最終的な推論を行う。過学習する傾向があるが、理解しやすい。木の深さや葉の数などを適切に決める必要がある。
<img src="11.png">


In [ ]:
from sklearn.tree import DecisionTreeClassifier as DTC
tree = DTC()
tree.fit(x_train, y_train)
print("Decision Tree score:", tree.score(x_test, y_test))

In [ ]:
pred_tree = tree.predict(x_test)
error = 0
for i in range(y_test.shape[0]):
    if pred_tree[i] != y_test[i]:
        error += 1
        print("predict:{}\t correct:{}".format(pred_tree[i], y_test[i]))
        plt.imshow(x_test[i].reshape(28, 28), cmap="gray")
        plt.axis("off")
        plt.show()
        if error >= 10:
            break

## Random Forest  
決定木を大量に用いて推論を平均する。決定木１本では過学習しやすいが、大量の過学習した木を集め、平均を取れば、テストデータに対しても高い精度を出すことができる。このような手法を<b>アンサンブル学習</b>という。
<img src="RandomForest.png">

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC
forest = RFC()
forest.fit(x_train, y_train)
print("Random Forest score:", forest.score(x_test, y_test))

In [ ]:
pred_RF = forest.predict(x_test)
error = 0
for i in range(y_test.shape[0]):
    if pred_RF[i] != y_test[i]:
        error += 1
        print("predict:{}\t correct:{}".format(pred_RF[i], y_test[i]))
        plt.imshow(x_test[i].reshape(28, 28), cmap="gray")
        plt.axis("off")
        plt.show()
        if error >= 10:
            break

## Neural Network
いわゆるディープラーニング。大量のニューロンと呼ばれるものを用いて段階的に精度を上げていく。画像認識や自然言語処理に強いが、いくつかのパラメータをうまく設定する必要がある。
<img src="neural_net.png">

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.utils import np_utils
#60000*784の訓練データandテストデータを60000*28*28に整形（NNは画像（２次元配列）として読み込むことができる）
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
#ニューラルネットワークモデルの生成
model = Sequential()    #インスタンス生成
model.add(Conv2D(16, (3, 3), input_shape=(28, 28, 1), activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(10, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])
model.summary()

print(x_train)

In [ ]:
#学習
history = model.fit(x_train, y_train, batch_size=1000, epochs=20, verbose=1, validation_split=0.1)
pred_NN = model.predict(x_test)
pred_NN = np.where(pred_NN < 0.5, 0, 1)

In [ ]:
for i in range(y_test.shape[0]):
    if not np.allclose(pred_NN[i], y_test[i]):
        error += 1
        print("predict:{}\t correct:{}".format(np.argmax(pred_NN[i]+1), np.argmax(y_test[i]+1)))
        plt.imshow(x_test[i].reshape(28, 28), cmap="gray")
        plt.axis("off")
        plt.show()
        if error >= 10:
            break